1.このSprintについて

Sprintの目的
アンサンブル学習について理解する

どのように学ぶか
スクラッチでアンサンブル学習の各種手法を実装していきます。



2.アンサンブル学習

3種類のアンサンブル学習をスクラッチ実装していきます。そして、それぞれの効果を小さめのデータセットで確認します。


ブレンディング
バギング
スタッキング

小さなデータセットの用意
以前も利用した回帰のデータセットを用意します。


House Prices: Advanced Regression Techniques


この中のtrain.csvをダウンロードし、目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使います。


train.csvを学習データ8割、検証データ2割に分割してください。


scikit-learn
単一のモデルはスクラッチ実装ではなく、scikit-learnなどのライブラリの使用を推奨します。


sklearn.linear_model.LinearRegression — scikit-learn 0.21.3 documentation


sklearn.svm.SVR — scikit-learn 0.21.3 documentation


sklearn.tree.DecisionTreeRegressor — scikit-learn 0.21.3 documentation




3.ブレンディング

【問題1】ブレンディングのスクラッチ実装
ブレンディング をスクラッチ実装し、単一モデルより精度があがる例を 最低3つ 示してください。精度があがるとは、検証データに対する平均二乗誤差（MSE）が小さくなることを指します。


ブレンディングとは
ブレンディングとは、N個の多様なモデルを独立して学習させ、推定結果を重み付けした上で足し合わせる方法です。最も単純には平均をとります。多様なモデルとは、以下のような条件を変化させることで作り出すものです。


手法（例：線形回帰、SVM、決定木、ニューラルネットワークなど）
ハイパーパラメータ（例：SVMのカーネルの種類、重みの初期値など）
入力データの前処理の仕方（例：標準化、対数変換、PCAなど）

重要なのはそれぞれのモデルが大きく異なることです。


回帰問題でのブレンディングは非常に単純であるため、scikit-learnには用意されていません。


《補足》


分類問題の場合は、多数決を行います。回帰問題に比べると複雑なため、scikit-learnにはVotingClassifierが用意されています。


sklearn.ensemble.VotingClassifier — scikit-learn 0.21.3 documentation



In [2]:
import pandas as pd

# import os
# os.listdir()
# print(os.pwd)
df = pd.read_csv('/content/drive/MyDrive/DIC/train.csv')
print(df)

        Id  MSSubClass MSZoning  ...  SaleType  SaleCondition SalePrice
0        1          60       RL  ...        WD         Normal    208500
1        2          20       RL  ...        WD         Normal    181500
2        3          60       RL  ...        WD         Normal    223500
3        4          70       RL  ...        WD        Abnorml    140000
4        5          60       RL  ...        WD         Normal    250000
...    ...         ...      ...  ...       ...            ...       ...
1455  1456          60       RL  ...        WD         Normal    175000
1456  1457          20       RL  ...        WD         Normal    210000
1457  1458          70       RL  ...        WD         Normal    266500
1458  1459          20       RL  ...        WD         Normal    142125
1459  1460          20       RL  ...        WD         Normal    147500

[1460 rows x 81 columns]


In [3]:
x=df.loc[:,['GrLivArea','YearBuilt']]
print(type(x))
print(x)

target = df.loc[:, ['SalePrice']]
print(target[:])
print(type(target))

<class 'pandas.core.frame.DataFrame'>
      GrLivArea  YearBuilt
0          1710       2003
1          1262       1976
2          1786       2001
3          1717       1915
4          2198       2000
...         ...        ...
1455       1647       1999
1456       2073       1978
1457       2340       1941
1458       1078       1950
1459       1256       1965

[1460 rows x 2 columns]
      SalePrice
0        208500
1        181500
2        223500
3        140000
4        250000
...         ...
1455     175000
1456     210000
1457     266500
1458     142125
1459     147500

[1460 rows x 1 columns]
<class 'pandas.core.frame.DataFrame'>


In [4]:
#線形回帰
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import pandas as pd
import pprint
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
X=x
#print(df.sample(n=3))
np.random.seed(0)
X = X.sample(n=500)#サンプル数を5００へ絞り込み
X=X.values


X=X.tolist()
scaler = StandardScaler()
scaler.fit(X)
X=scaler.transform(X)
# print(X)

Y = target
np.random.seed(0)
Y = Y.sample(n=500)#サンプル数を5００へ絞り込み
Y=Y.values
# print(Y.head)

# トレーニング・テストデータ分割
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size= 0.8, test_size=0.2, random_state=42)

# scaler.fit(X_train)
# X_train=scaler.transform(X_train)
lr = LinearRegression()
lr.fit(X_train, Y_train)

# 予測　
# X_test=scaler.transform(X_test) #MSE向上させる為に標準化追加
Y_pred = lr.predict(X_test)

mse1_1 = mean_squared_error(Y_test, Y_pred)
print("線形回帰MSE1_1 = %.2f" % (mse1_1))


#SVM
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import numpy as np
n_samples, n_features = 10, 5



regr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))

regr.fit(X_train, Y_train)


# 予測　
Y_pred2 = regr.predict(X_test)
mse2_1 = mean_squared_error(Y_test, Y_pred2)
print("SVM_MSE2_1 = %.2f" % (mse2_1))



#決定木
from sklearn import tree

clf = tree.DecisionTreeRegressor()
# # 対数変換を行う
# X_train = np.log(X_train)
clf = clf.fit(X_train, Y_train)

# 予測　
Y_pred3 = clf.predict(X_test)
mse3_1 = mean_squared_error(Y_test, Y_pred3)
print("決定木MSE3_1 = %.2f" % (mse3_1))



線形回帰MSE1_1 = 1586822023.36
SVM_MSE2_1 = 4390908443.07
決定木MSE3_1 = 3191872660.18


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [5]:
#上記のモデル（線形回帰、SVM、決定木）で推定した結果を重み付けした上で足し合わせる方法です。最も単純には平均をとります。求める平均値をAve１と置く
Ave1 = (mse1_1+mse2_1+mse3_1)/3
print(Ave1)

3056534375.5368657


In [6]:
#決定木の標準化追加
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import pandas as pd
import pprint
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
X=x
#print(df.sample(n=3))
np.random.seed(0)
X = X.sample(n=500)#サンプル数を5００へ絞り込み
X=X.values


X=X.tolist()
scaler = StandardScaler()
scaler.fit(X)
X=scaler.transform(X)
# print(X)

Y = target
np.random.seed(0)
Y = Y.sample(n=500)#サンプル数を5００へ絞り込み
Y=Y.values
# print(Y.head)

# トレーニング・テストデータ分割
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size= 0.8, test_size=0.2, random_state=42)


lr = LinearRegression()
lr.fit(X_train, Y_train)

# 予測　
# X_test=scaler.transform(X_test) #MSE向上させる為に標準化追加
Y_pred = lr.predict(X_test)

mse1_2 = mean_squared_error(Y_test, Y_pred)
print("線形回帰MSE1_2 = %.2f" % (mse1_2))


#SVM
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import numpy as np
n_samples, n_features = 10, 5



regr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
# scaler.fit(X_train)
# X_train=scaler.transform(X_train)
regr.fit(X_train, Y_train)


# 予測　
Y_pred2 = regr.predict(X_test)
mse2_2 = mean_squared_error(Y_test, Y_pred2)
print("SVM_MSE2_2 = %.2f" % (mse2_2))



#決定木
from sklearn import tree

clf = tree.DecisionTreeRegressor()
# # 対数変換を行う
# X_train = np.log(X_train)
scaler.fit(X_train)
X_train=scaler.transform(X_train)
clf = clf.fit(X_train, Y_train)

# 予測　
Y_pred3 = clf.predict(X_test)
mse3_2 = mean_squared_error(Y_test, Y_pred3)
print("決定木MSE3_2 = %.2f" % (mse3_2))



線形回帰MSE1_2 = 1586822023.36
SVM_MSE2_2 = 4390908443.07
決定木MSE3_2 = 2904205466.29


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [7]:
#決定木の標準化でMSE向上
Ave2 = (mse1_2+mse2_2+mse3_2)/3
print(Ave2)

2960645310.9068656


単一モデルの
（線形回帰、SVM、決定木）のMSE平均値：3056534375.5368657

決定木の標準化追加したフルスクラッチ
（線形回帰、SVM、決定木）のMSE平均値：2960645310.9068656

よって、例１は、向上した

In [8]:
#線形回帰の標準化追加
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import pandas as pd
import pprint
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
X=x
#print(df.sample(n=3))
np.random.seed(0)
X = X.sample(n=500)#サンプル数を5００へ絞り込み
X=X.values


X=X.tolist()
scaler = StandardScaler()
scaler.fit(X)
X=scaler.transform(X)
# print(X)

Y = target
np.random.seed(0)
Y = Y.sample(n=500)#サンプル数を5００へ絞り込み
Y=Y.values
# print(Y.head)

# トレーニング・テストデータ分割
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size= 0.8, test_size=0.2, random_state=42)

scaler.fit(X_train)
X_train=scaler.transform(X_train)
lr = LinearRegression()
scaler.fit(X_train)
X_train=scaler.transform(X_train)
lr.fit(X_train, Y_train)

# 予測　
# X_test=scaler.transform(X_test) #MSE向上させる為に標準化追加
Y_pred = lr.predict(X_test)

mse1_3 = mean_squared_error(Y_test, Y_pred)
print("線形回帰MSE1_3 = %.2f" % (mse1_3))


#SVM
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import numpy as np
n_samples, n_features = 10, 5



regr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))

regr.fit(X_train, Y_train)


# 予測　
Y_pred2 = regr.predict(X_test)
mse2_3 = mean_squared_error(Y_test, Y_pred2)
print("SVM_MSE2_3 = %.2f" % (mse2_3))



#決定木
from sklearn import tree

clf = tree.DecisionTreeRegressor()
# # 対数変換を行う
# X_train = np.log(X_train)
clf = clf.fit(X_train, Y_train)

# 予測　
Y_pred3 = clf.predict(X_test)
mse3_3 = mean_squared_error(Y_test, Y_pred3)
print("決定木MSE3_3 = %.2f" % (mse3_3))



線形回帰MSE1_3 = 1606608715.57
SVM_MSE2_3 = 4390905192.65
決定木MSE3_3 = 2904205466.29


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [9]:
#線形回帰の標準化でMSE向上
Ave3 = (mse1_3+mse2_3+mse3_3)/3
print(Ave3)

2967239791.502683


単一モデルのフルスクラッチ
（線形回帰、SVM、決定木）のMSE平均値：3056534375.5368657

線形回帰の対数変換追加したフルスクラッチ
（線形回帰、SVM、決定木）のMSE平均値：2967239791.502683

よって、例２は、向上した

In [10]:
#SVMのカーネル関数変更
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import pandas as pd
import pprint
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
X=x
#print(df.sample(n=3))
np.random.seed(0)
X = X.sample(n=500)#サンプル数を5００へ絞り込み
X=X.values


X=X.tolist()
scaler = StandardScaler()
scaler.fit(X)
X=scaler.transform(X)
# print(X)

Y = target
np.random.seed(0)
Y = Y.sample(n=500)#サンプル数を5００へ絞り込み
Y=Y.values
# print(Y.head)

# トレーニング・テストデータ分割
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size= 0.8, test_size=0.2, random_state=42)



lr = LinearRegression()
lr.fit(X_train, Y_train)

# 予測　
# X_test=scaler.transform(X_test) #MSE向上させる為に標準化追加
Y_pred = lr.predict(X_test)

mse1_4 = mean_squared_error(Y_test, Y_pred)
print("線形回帰MSE1_4 = %.2f" % (mse1_4))


#SVM
from sklearn.svm import SVR
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import numpy as np
n_samples, n_features = 10, 5


svm = SVC(kernel = 'rbf')
# regr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))

svm.fit(X_train, Y_train)


# 予測　
Y_pred2 = regr.predict(X_test)
mse2_4 = mean_squared_error(Y_test, Y_pred2)
print("SVM_MSE2_4 = %.2f" % (mse2_4))



#決定木
from sklearn import tree

clf = tree.DecisionTreeRegressor()
# # 対数変換を行う
# X_train = np.log(X_train)
clf = clf.fit(X_train, Y_train)

# 予測　
Y_pred3 = clf.predict(X_test)
mse3_4 = mean_squared_error(Y_test, Y_pred3)
print("決定木MSE3_4 = %.2f" % (mse3_4))



線形回帰MSE1_4 = 1586822023.36


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVM_MSE2_4 = 4390905192.65
決定木MSE3_4 = 3191872660.18


In [11]:
#SVMのカーネルモデルをrbfに変更してMSE向上
Ave4 = (mse1_4+mse2_4+mse3_4)/3
print(Ave4)

3056533292.064443


単一モデルのフルスクラッチ
（線形回帰、SVM、決定木）のMSE平均値：3056534375.5368657

SVMのカーネルモデルをrbfに変更してMSE向上したフルスクラッチ
（線形回帰、SVM、決定木）のMSE平均値：3056533292.064443

よって、例３は、向上した

4.バギング

【問題2】バギングのスクラッチ実装
バギング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。


バギングとは
バギングは入力データの選び方を多様化する方法です。訓練データから重複を許した上でランダムに抜き出すことで、N種類のサブセット（ ブートストラップサンプル ）を作り出します。それらによってモデルをN個学習し、推定結果の平均をとります。ブレンディングと異なり、それぞれの重み付けを変えることはありません。


sklearn.model_selection.train_test_split — scikit-learn 0.21.3 documentation


推定結果の平均をとる部分はブレンディングと同様の実装になります。




In [12]:

import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import pandas as pd
import pprint
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import resample

X=x
#print(df.sample(n=3))
np.random.seed(0)
X = X.sample(n=500)#サンプル数を5００へ絞り込み
X=X.values


X=X.tolist()
scaler = StandardScaler()
scaler.fit(X)
X=scaler.transform(X)
# print(X)

Y = target
np.random.seed(0)
Y = Y.sample(n=500)#サンプル数を5００へ絞り込み
Y=Y.values
# print(Y.head)

# # トレーニング・テストデータ分割
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size= 0.8, test_size=0.2, random_state=42)



lr = LinearRegression()
lr.fit(X_train, Y_train)

# 予測　
# X_test=scaler.transform(X_test) #MSE向上させる為に標準化追加
Y_pred = lr.predict(X_test)

mse1_1 = mean_squared_error(Y_test, Y_pred)
print("線形回帰MSE1_1 = %.2f" % (mse1_1))





線形回帰MSE1_1 = 1586822023.36


In [13]:
#線形回帰N種類のサブセット
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import pandas as pd
import pprint
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import resample

X=x
#print(df.sample(n=3))
np.random.seed(0)
X = X.sample(n=500)#サンプル数を5００へ絞り込み
X=X.values


X=X.tolist()
scaler = StandardScaler()
scaler.fit(X)
X=scaler.transform(X)
# print(X)

Y = target
np.random.seed(0)
Y = Y.sample(n=500)#サンプル数を5００へ絞り込み
Y=Y.values
# print(Y.head)

# # トレーニング・テストデータ分割
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size= 0.8, test_size=0.2, random_state=42)


#訓練データから重複を許した上でランダムに抜き出すことで、N種類のサブセット（ ブートストラップサンプル ）を作る
# N:再標本化の回数 SIZE:再標本化されたもののサンプルサイズ
N = 10
SIZE = len(X_train)
mse1_1_list=0

# bootstrap sampling
for i in range(N):
    resampled_data_x, resampled_data_y = resample(X_train,Y_train, 
                                                   n_samples = SIZE)
    lr = LinearRegression()
    lr.fit(resampled_data_x, resampled_data_y)

    # 予測　
    # X_test=scaler.transform(X_test) #MSE向上させる為に標準化追加
    Y_pred = lr.predict(X_test)
    mse1_1 = mean_squared_error(Y_test, Y_pred)

    print("------%d resample ------"% i)
    print("線形回帰MSE1_1 = %.2f" % (mse1_1))
    # print(resampled_data_x)
    # print(resampled_data_y)
    mse1_1_list+=mse1_1

#N回のMSE平均値は、
mse1_1_final = mse1_1_list/N
print("mse1_1_final= %.2f" % (mse1_1_final))








------0 resample ------
線形回帰MSE1_1 = 1696970231.76
------1 resample ------
線形回帰MSE1_1 = 1469995038.98
------2 resample ------
線形回帰MSE1_1 = 2480264230.31
------3 resample ------
線形回帰MSE1_1 = 1854831480.21
------4 resample ------
線形回帰MSE1_1 = 1719834400.10
------5 resample ------
線形回帰MSE1_1 = 1500928467.87
------6 resample ------
線形回帰MSE1_1 = 1969040961.94
------7 resample ------
線形回帰MSE1_1 = 1453575958.00
------8 resample ------
線形回帰MSE1_1 = 2138819444.46
------9 resample ------
線形回帰MSE1_1 = 1595176817.69
mse1_1_final= 1787943703.13


In [14]:
#SVM
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import pandas as pd
import pprint
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import resample

X=x
#print(df.sample(n=3))
np.random.seed(0)
X = X.sample(n=500)#サンプル数を5００へ絞り込み
X=X.values


X=X.tolist()
scaler = StandardScaler()
scaler.fit(X)
X=scaler.transform(X)
# print(X)

Y = target
np.random.seed(0)
Y = Y.sample(n=500)#サンプル数を5００へ絞り込み
Y=Y.values
# print(Y.head)

# # トレーニング・テストデータ分割
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size= 0.8, test_size=0.2, random_state=42)


#SVM
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import numpy as np
n_samples, n_features = 10, 5



regr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
# scaler.fit(X_train)
# X_train=scaler.transform(X_train)
regr.fit(X_train, Y_train)


# 予測　
Y_pred2 = regr.predict(X_test)
mse2_1 = mean_squared_error(Y_test, Y_pred2)
print("SVM_MSE2_1 = %.2f" % (mse2_1))





SVM_MSE2_1 = 4390908443.07


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [15]:
#SVM_N種類のサブセット（ ブートストラップサンプル ）
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import pandas as pd
import pprint
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import resample

X=x
#print(df.sample(n=3))
np.random.seed(0)
X = X.sample(n=500)#サンプル数を5００へ絞り込み
X=X.values


X=X.tolist()
scaler = StandardScaler()
scaler.fit(X)
X=scaler.transform(X)
# print(X)

Y = target
np.random.seed(0)
Y = Y.sample(n=500)#サンプル数を5００へ絞り込み
Y=Y.values
# print(Y.head)

# # トレーニング・テストデータ分割
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size= 0.8, test_size=0.2, random_state=42)


#訓練データから重複を許した上でランダムに抜き出すことで、N種類のサブセット（ ブートストラップサンプル ）を作る
# N:再標本化の回数 SIZE:再標本化されたもののサンプルサイズ
N = 10
SIZE = len(X_train)
mse2_2_list=0

# bootstrap sampling
for i in range(N):
    resampled_data_x, resampled_data_y = resample(X_train,Y_train, 
                                                   n_samples = SIZE)
    regr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
    # scaler.fit(X_train)
    # X_train=scaler.transform(X_train)
    regr.fit(resampled_data_x, resampled_data_y)


    # 予測　
    Y_pred2 = regr.predict(X_test)
    mse2_2 = mean_squared_error(Y_test, Y_pred2)

    print("------%d resample ------"% i)
    print("SVM_MSE2_2 = %.2f" % (mse2_2))
    # print(resampled_data_x)
    # print(resampled_data_y)
    mse2_2_list+=mse2_2

#N回のMSE平均値は、
mse2_2_final = mse2_2_list/N
print("mse2_2_final= %.2f" % (mse2_2_final))








------0 resample ------
SVM_MSE2_2 = 4451485869.01


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

------1 resample ------
SVM_MSE2_2 = 4307229024.49
------2 resample ------
SVM_MSE2_2 = 4504339287.61
------3 resample ------
SVM_MSE2_2 = 4333948365.91
------4 resample ------
SVM_MSE2_2 = 4390995057.46
------5 resample ------
SVM_MSE2_2 = 4390936086.14
------6 resample ------
SVM_MSE2_2 = 4390979335.27
------7 resample ------
SVM_MSE2_2 = 4476168312.53
------8 resample ------
SVM_MSE2_2 = 4450602863.02
------9 resample ------
SVM_MSE2_2 = 4605268745.96
mse2_2_final= 4430195294.74


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

In [16]:
#決定木
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import pandas as pd
import pprint
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import resample

X=x
#print(df.sample(n=3))
np.random.seed(0)
X = X.sample(n=500)#サンプル数を5００へ絞り込み
X=X.values


X=X.tolist()
scaler = StandardScaler()
scaler.fit(X)
X=scaler.transform(X)
# print(X)

Y = target
np.random.seed(0)
Y = Y.sample(n=500)#サンプル数を5００へ絞り込み
Y=Y.values
# print(Y.head)

# # トレーニング・テストデータ分割
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size= 0.8, test_size=0.2, random_state=42)




#決定木
from sklearn import tree

clf = tree.DecisionTreeRegressor()
# # 対数変換を行う
# X_train = np.log(X_train)
scaler.fit(X_train)
X_train=scaler.transform(X_train)
clf = clf.fit(X_train, Y_train)

# 予測　
Y_pred3 = clf.predict(X_test)
mse3_1 = mean_squared_error(Y_test, Y_pred3)
print("決定木MSE3_1 = %.2f" % (mse3_1))







決定木MSE3_1 = 2897938419.21


In [17]:
#決定木_N種類のサブセット（ ブートストラップサンプル ）
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import pandas as pd
import pprint
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import resample

X=x
#print(df.sample(n=3))
np.random.seed(0)
X = X.sample(n=500)#サンプル数を5００へ絞り込み
X=X.values


X=X.tolist()
scaler = StandardScaler()
scaler.fit(X)
X=scaler.transform(X)
# print(X)

Y = target
np.random.seed(0)
Y = Y.sample(n=500)#サンプル数を5００へ絞り込み
Y=Y.values
# print(Y.head)

# # トレーニング・テストデータ分割
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size= 0.8, test_size=0.2, random_state=42)


#訓練データから重複を許した上でランダムに抜き出すことで、N種類のサブセット（ ブートストラップサンプル ）を作る
# N:再標本化の回数 SIZE:再標本化されたもののサンプルサイズ
N = 10
SIZE = len(X_train)
mse3_2_list=0

# bootstrap sampling
for i in range(N):
    resampled_data_x, resampled_data_y = resample(X_train,Y_train, 
                                                   n_samples = SIZE)
    lr = LinearRegression()
    lr.fit(resampled_data_x, resampled_data_y)

    
    #決定木
    from sklearn import tree

    clf = tree.DecisionTreeRegressor()
    # # 対数変換を行う
    # X_train = np.log(X_train)
    scaler.fit(X_train)
    X_train=scaler.transform(X_train)
    clf = clf.fit(X_train, Y_train)

    # 予測　
    Y_pred3 = clf.predict(X_test)
    mse3_2 = mean_squared_error(Y_test, Y_pred3)
    print("決定木MSE3_2 = %.2f" % (mse3_2))
        # print(resampled_data_x)
        # print(resampled_data_y)
    mse3_2_list+=mse1_1

#N回のMSE平均値は、
mse3_2_final = mse3_2_list/N
print("mse3_2_final= %.2f" % (mse3_2_final))








決定木MSE3_2 = 3113485747.46
決定木MSE3_2 = 3047505218.54
決定木MSE3_2 = 2941046941.29
決定木MSE3_2 = 3177266605.77
決定木MSE3_2 = 2879429945.77
決定木MSE3_2 = 3117070109.21
決定木MSE3_2 = 3104922944.02
決定木MSE3_2 = 3031541192.46
決定木MSE3_2 = 3122524244.54
決定木MSE3_2 = 2945133445.77
mse3_2_final= 1595176817.69


線形回帰_元のMSE：1586822023.36 
バギングのMSE結果：1787943703.13

SVM_元のMSE：4390908443.07 
バギングのMSE結果：4430195294.74

決定木_元のMSE：2897938419.21 
バギングのMSE結果：1595176817.69

よって、決定木だけがバギングでMSEが向上した。

5.スタッキング

【問題3】スタッキングのスクラッチ実装
スタッキング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。


スタッキングとは
スタッキングの手順は以下の通りです。最低限ステージ0とステージ1があればスタッキングは成立するため、それを実装してください。まずは 
K
0
=
3
,
M
0
=
2
 程度にします。


《学習時》


（ステージ 
0
 ）


訓練データを 
K
0
 個に分割する。
分割した内の 
(
K
0
−
1
)
 個をまとめて訓練データ、残り 
1
 個を推定用データとする組み合わせが 
K
0
 個作れる。
あるモデルのインスタンスを 
K
0
 個用意し、異なる訓練データを使い学習する。
それぞれの学習済みモデルに対して、使っていない残り 
1
 個の推定用データを入力し、推定値を得る。（これをブレンドデータと呼ぶ）
さらに、異なるモデルのインスタンスも 
K
0
 個用意し、同様のことを行う。モデルが 
M
0
 個あれば、 
M
0
 個のブレンドデータが得られる。

（ステージ 
n
 ）


ステージ 
n
−
1
 のブレンドデータを
M
n
−
1
 次元の特徴量を持つ訓練データと考え、 
K
n
 個に分割する。以下同様である。

（ステージ 
N
 ）＊最後のステージ


ステージ 
N
−
1
 の 
M
N
−
1
 個のブレンドデータを
M
N
−
1
 次元の特徴量の入力として、1種類のモデルの学習を行う。これが最終的な推定を行うモデルとなる。

《推定時》


（ステージ 
0
 ）


テストデータを 
K
0
×
M
0
 個の学習済みモデルに入力し、
K
0
×
M
0
 個の推定値を得る。これを 
K
0
 の軸で平均値を求め 
M
0
 次元の特徴量を持つデータを得る。（ブレンドテストと呼ぶ）

（ステージ 
n
 ）


ステージ 
n
−
1
 で得たブレンドテストを 
K
n
×
M
n
 個の学習済みモデルに入力し、
K
n
×
M
n
 個の推定値を得る。これを 
K
n
 の軸で平均値を求め 
M
0
 次元の特徴量を持つデータを得る。（ブレンドテストと呼ぶ）

（ステージ 
N
 ）＊最後のステージ


ステージ 
N
−
1
 で得たブレンドテストを学習済みモデルに入力し、推定値を得る。

学習時

In [18]:
# クロスバリデーション
from sklearn.model_selection import KFold
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor as RFR # ★
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVR  # ★
from sklearn.linear_model import LinearRegression as LR # ★
from sklearn.tree import DecisionTreeRegressor as DTR # ★

X_train_list = []
X_test_list = []
y_train_list = []
y_test_list = []
model_ins = []
i = 0
y_brend1 = None
y_brend2 = None
y_brend3 = None
y_brend4 = None

model_ins1 = []
model_ins2 = []


# << stage 0 >> 

# 分割数
K0= 3

# モデル数
M0 = 2

ss_X=X
y=Y

# クロスバリデーション
kf = KFold(n_splits = K0)
kf.get_n_splits(ss_X)

# 変数初期化
y_brend = np.zeros(ss_X.shape[0] * M0).reshape(ss_X.shape[0], M0).reshape(-1, M0)
model_ins_0 = np.empty(K0 * M0).reshape(K0, M0)
model_ins_0 = model_ins_0.astype(np.object)

# インスタンス作成
for ins in range(K0):
    
    model_ins_0[ins,0] = SVR()
    model_ins_0[ins,1] = RFR()


# データ分割
for i, (train_index, test_index) in enumerate(kf.split(ss_X)):
    print("i = ", i)
    #print("TRAIN:", train_index, "TEST:", test_index)

        
    X_train, X_test = ss_X[train_index], ss_X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    X_train_list.append(X_train)
    X_test_list.append(X_test)
    y_train_list.append(y_train)
    y_test_list.append(y_test)
    print("X_train.shape = {}".format(X_train.shape))
    print("X_test.shape = {}".format(X_train.shape))
    print("y_train.shape = {}".format(y_train.shape))
    print("y_test.shape = {}".format(y_test.shape))
    
    
    for j in range(M0):
        # 学習
        model_ins_0[i,j].fit(X_train, y_train)

        # 推定
        y_brend[test_index, j] = model_ins_0[i,j].predict(X_test).flatten()
    
#print("★★★y_brend = \n{}".format(y_brend))
#print("★★★y_brend.shape = {}".format(y_brend.shape))


# << stage n >> 
Kn = 3
Mn = 2
n_cnt = 1
kf = KFold(n_splits = Kn)
kf.get_n_splits(X)

# 変数初期化
model_ins_n = np.empty(Kn * Mn).reshape(Kn, Mn)
model_ins_n = model_ins_n.astype(np.object)


# インスタンス作成
for ins in range(K0):
    
    model_ins_n[ins,0] = LR()
    model_ins_n[ins,1] = DTR()

# n回ループ
for n in range(n_cnt):
    for i, (train_index, test_index) in enumerate(kf.split(y_brend)):

        X_train, X_test = y_brend[train_index], y_brend[test_index]
        y_train, y_test = y[train_index], y[test_index]
        X_train_list.append(X_train)
        X_test_list.append(X_test)
        y_train_list.append(y_train)
        y_test_list.append(y_test)


        for j in range(Mn):
            # 学習
            model_ins_n[i,j].fit(X_train, y_train)

            # 推定
            y_brend[test_index, j] = model_ins_n[i,j].predict(X_test).flatten()


# << Last Stage >>
Kl = 3
Ml = 1
n_cnt = 1
kf = KFold(n_splits = Kl)
kf.get_n_splits(X)

# 変数初期化
y_brend_fin = np.zeros(ss_X.shape[0] * Ml).reshape(ss_X.shape[0], Ml).reshape(-1, Ml)
model_ins_l = np.empty(Kl * Ml).reshape(Kl, Ml)
model_ins_l = model_ins_l.astype(np.object)


# インスタンス作成
for ins in range(Kl):
    # model_ins_l[ins,0] = Lasso(alpha=0.1)
    model_ins_l[ins,0] = RFR()
    # model_ins_l[ins,0] = GaussianNB()
    
for i, (train_index, test_index) in enumerate(kf.split(y_brend)):

    X_train, X_test = y_brend[train_index], y_brend[test_index]
    y_train, y_test = y[train_index], y[test_index]
    X_train_list.append(X_train)
    X_test_list.append(X_test)
    y_train_list.append(y_train)
    y_test_list.append(y_test)

    # 学習
    model_ins_l[i,0].fit(X_train, y_train)

    # 推定
    y_brend_fin[test_index, 0] = model_ins_l[i,0].predict(X_test).flatten()
    
    
print("y_brend_fin.shape = ", y_brend_fin.shape)

i =  0
X_train.shape = (333, 2)
X_test.shape = (333, 2)
y_train.shape = (333, 1)
y_test.shape = (167, 1)


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


i =  1
X_train.shape = (333, 2)
X_test.shape = (333, 2)
y_train.shape = (333, 1)
y_test.shape = (167, 1)
i =  2
X_train.shape = (334, 2)
X_test.shape = (334, 2)
y_train.shape = (334, 1)
y_test.shape = (166, 1)


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/usr/local/lib/python3.7/dist-packages

y_brend_fin.shape =  (500, 1)


推定時

In [28]:
# << stage 0 >> 
predict_model_0 = np.zeros(ss_X.shape[0] * M0).reshape(ss_X.shape[0], -1)
for m in range(M0):
    predict_instance_0 = np.zeros(ss_X.shape[0] * K0).reshape(ss_X.shape[0], -1)
    
    for k in range(K0):
        # 推定
        predict_instance_0[:,k] = model_ins_0[k,m].predict(ss_X)
        
    predict_model_0[:,m] = np.mean(predict_instance_0, axis = 1).flatten()

# << stage n >> 
# n回ループ
for n in range(n_cnt):
    predict_model_n = np.zeros(ss_X.shape[0] * Mn).reshape(ss_X.shape[0], -1)
    predict_model_n = predict_model_0
    for m in range(M0):
        predict_instance_n = np.zeros(ss_X.shape[0] * Kn).reshape(ss_X.shape[0] , -1)

        for k in range(K0):
            # predict_instance_n[:,k] = model_ins_n[k,m].predict(predict_model_n)
            # predict_instance_n.reshape=predict_instance_n.reshape(-1,1)
            predict_instance_n = model_ins_n[k,m].predict(predict_model_n)
        predict_instance_n=predict_instance_n.reshape(-1,1)   
        # predict_model_n[:,m] = np.mean(predict_instance_n, axis = 1).flatten()
        predict_model_n[:,m] = np.mean(predict_instance_n, axis = 1).flatten()
        
# << Last Stage >>
# 推定
y_predict_fin = model_ins_l[i,0].predict(predict_model_n).flatten()
y_predict_fin.reshape(ss_X.shape[0], -1)

print("y_predict_fin.shape", y_predict_fin.shape)

y_predict_fin.shape (500,)


In [33]:
# 評価
# MSE = mean_squared_error(y, y_predict_fin)
MSE = mean_squared_error(Y.flatten(), y_predict_fin)
MSE

7232915507.75797

問題３は、あまり精度が出なかった。